In [119]:
import pandas as pd
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
# ignore Deprecation Warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.utils import plot_model

In [120]:
ingr_path = 'data/flavor_network_data/ingr_comp/ingr_info.tsv'
ingr_df = pd.read_csv(ingr_path, delimiter='\t')
ingr_df.head()

,# id,ingredient name,category
0,0,magnolia_tripetala,flower
1,1,calyptranthes_parriculata,plant
2,2,chamaecyparis_pisifera_oil,plant derivative
3,3,mackerel,fish/seafood
4,4,mimusops_elengi_flower,flower


In [121]:
ingr_df.shape

(1530, 3)

In [122]:
ingr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1530 entries, 0 to 1529
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   # id             1530 non-null   int64 
 1   ingredient name  1530 non-null   object
 2   category         1530 non-null   object
dtypes: int64(1), object(2)
memory usage: 36.0+ KB


In [123]:
ingr_df['ingredient name'] = ingr_df['ingredient name'].str.lower()
ingr_df['category'] = ingr_df['category'].str.lower()

In [124]:
special_characters = set()


for sublist in ingr_df['ingredient name']:
    for value in sublist:
      
        for char in value:
            if not char.isalnum():  
                special_characters.add(char)


print("special charac in column 'ingredient name':")
print(special_characters)

special charac in column 'ingredient name':
{'_'}


In [125]:
special_characters = set()


for sublist in ingr_df['category']:
    for value in sublist:
      
        for char in value:
            if not char.isalnum():  
                special_characters.add(char)


print("special charac in column 'category':")
print(special_characters)

special charac in column 'category':
{'/', ' '}


In [126]:
unique_ingredients = ingr_df['ingredient name'].unique()

new_ingredient = pd.DataFrame(columns=unique_ingredients)

In [127]:
new_ingredient.head(2)

,magnolia_tripetala,calyptranthes_parriculata,chamaecyparis_pisifera_oil,mackerel,mimusops_elengi_flower,hyssop,buchu,black_pepper,eryngium_poterium_oil,peanut_butter,...,rue_oil,roasted_coconut,cajeput_oil,star_anise,feijoa,tobacco_oil,cinnamon_leaf,green_tea,artemisia_porrecta_oil,munster_cheese


In [128]:
recipe_df = pd.read_csv('data/full_dataset.csv')

In [129]:
recipe_df.shape

(2231142, 7)

In [130]:
recipe_df.head(1)

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."


In [131]:
recipe_ing_dir=recipe_df[['NER','directions']]

In [132]:
recipe_ing_dir.head(1)

,NER,directions
0,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...","[""In a heavy 2-quart saucepan, mix brown sugar..."


In [133]:
num_dfs = 200


chunk_size = len(recipe_ing_dir) // num_dfs


dfs = [recipe_ing_dir.iloc[i:i + chunk_size] for i in range(0, len(recipe_ing_dir), chunk_size)]


if len(dfs) > num_dfs:
    dfs[num_dfs-1] = pd.concat([dfs[num_dfs-1], dfs[num_dfs]], ignore_index=True)
    dfs = dfs[:num_dfs]


for i, df in enumerate(dfs):
    globals()[f'df_{i+1}'] = df


print(df_1.head())
print("------")
print(df_2.head())

                                                 NER   
0  ["brown sugar", "milk", "vanilla", "nuts", "bu...  \
1  ["beef", "chicken breasts", "cream of mushroom...   
2  ["frozen corn", "cream cheese", "butter", "gar...   
3  ["chicken", "chicken gravy", "cream of mushroo...   
4  ["peanut butter", "graham cracker crumbs", "bu...   

                                          directions  
0  ["In a heavy 2-quart saucepan, mix brown sugar...  
1  ["Place chipped beef on bottom of baking dish....  
2  ["In a slow cooker, combine all ingredients. C...  
3  ["Boil and debone chicken.", "Put bite size pi...  
4  ["Combine first four ingredients and press in ...  
------
                                                     NER   
11155  ["butter", "powdered sugar", "vanilla", "water...  \
11156  ["cucumbers", "salt", "sesame seeds", "sugar",...   
11157  ["sugar", "water", "white Karo syrup", "salt",...   
11158  ["flour", "baking soda", "salt", "baking powde...   
11159  ["banana", "sugar",

In [134]:
df_34.head()

,NER,directions
368115,"[""mushroom soup"", ""onion soup""]","[""Put roast in oven safe pan, frozen."", ""Blend..."
368116,"[""chicken livers"", ""ground meat"", ""parsley"", ""...","[""Ground chicken livers and mix with ground me..."
368117,"[""sugar"", ""margarine"", ""light corn syrup"", ""eg...","[""Preheat oven to 350\u00b0."", ""Beat together ..."
368118,"[""cream cheese"", ""butter"", ""egg yolks"", ""onion...","[""Work first four ingredients together."", ""Fol..."
368119,"[""flour"", ""sugar"", ""salt"", ""butter""]","[""Cream butter until light."", ""Add flour, suga..."


In [135]:
df_137.shape

(11155, 2)

In [136]:
df_137.head()

,NER,directions
1517080,"[""ground beef"", ""green peppers"", ""onion"", ""gar...","[""In a stockpot, cook the beef, green peppers ..."
1517081,"[""red potatoes"", ""links"", ""fresh kielbasa"", ""b...","[""Place potatoes in a large saucepan and cover..."
1517082,"[""pork loin chops"", ""vegetable oil"", ""butter"",...","[""In a large skillet, brown pork chops in oil ..."
1517083,"[""egg"", ""chicken broth"", ""butter"", ""cubes"", ""m...","[""In a large bowl, combine the egg, broth and ..."
1517084,"[""active dry yeast"", ""water"", ""caraway seeds"",...","[""In a small bowl, dissolve yeast in warm wate..."


In [137]:
def clean_ingredient(ingredient):
  # Remove special characters
  cleaned = re.sub(r'[^a-zA-Z0-9\s]', '', ingredient)
  cleaned = cleaned.replace(' ', '_')   ####aggiunto
  return cleaned.strip()# Function to process and clean the NER column
def process_and_clean_ner_column(ner_list):
  try:
    # Evaluate the string to a list if necessary
    if isinstance(ner_list, str):
      ner_list = eval(ner_list)
    # Ensure ner_list is actually a list
    if isinstance(ner_list, list):
      # Clean each ingredient in the list
      cleaned_ingredients = [clean_ingredient(ingredient) for ingredient in ner_list]
      return cleaned_ingredients
    else:
      return ner_list
  except Exception as e:
    print(f"Error processing list: {e}")
    return ner_list # Return the original list if there's an error
  # Apply the processing function to the NER column
df_137['NER'] = df_137['NER'].apply(process_and_clean_ner_column)
# Print the cleaned DataFrame
print(df_137)

                                                       NER   
1517080  [ground_beef, green_peppers, onion, garlic, to...  \
1517081  [red_potatoes, links, fresh_kielbasa, butter, ...   
1517082  [pork_loin_chops, vegetable_oil, butter, water...   
1517083  [egg, chicken_broth, butter, cubes, mushroom_s...   
1517084  [active_dry_yeast, water, caraway_seeds, salt,...   
...                                                    ...   
1528230  [frozen_rhubarb, cold_water, sugar, raspberry,...   
1528231  [Pastry, sugar, allpurpose, salt, fresh_pears,...   
1528232  [butter, sugar, milk, mint_Andes, marshmallow_...   
1528233  [bacon, broccoli_florets, onion, mushrooms, li...   
1528234  [ground_beef, onions, potatoes, green_pepper, ...   

                                                directions  
1517080  ["In a stockpot, cook the beef, green peppers ...  
1517081  ["Place potatoes in a large saucepan and cover...  
1517082  ["In a large skillet, brown pork chops in oil ...  
1517083  ["

C:\Users\cunia\AppData\Local\Temp\ipykernel_17060\2841042643.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_137['NER'] = df_137['NER'].apply(process_and_clean_ner_column)


In [138]:
df_137.head(2)

,NER,directions
1517080,"[ground_beef, green_peppers, onion, garlic, to...","[""In a stockpot, cook the beef, green peppers ..."
1517081,"[red_potatoes, links, fresh_kielbasa, butter, ...","[""Place potatoes in a large saucepan and cover..."


In [139]:
df_137['NER'].unique

<bound method Series.unique of 1517080    [ground_beef, green_peppers, onion, garlic, to...
1517081    [red_potatoes, links, fresh_kielbasa, butter, ...
1517082    [pork_loin_chops, vegetable_oil, butter, water...
1517083    [egg, chicken_broth, butter, cubes, mushroom_s...
1517084    [active_dry_yeast, water, caraway_seeds, salt,...
                                 ...                        
1528230    [frozen_rhubarb, cold_water, sugar, raspberry,...
1528231    [Pastry, sugar, allpurpose, salt, fresh_pears,...
1528232    [butter, sugar, milk, mint_Andes, marshmallow_...
1528233    [bacon, broccoli_florets, onion, mushrooms, li...
1528234    [ground_beef, onions, potatoes, green_pepper, ...
Name: NER, Length: 11155, dtype: object>

In [140]:

"""new_ingredient = new_ingredient.reindex(index=df_137.index).fillna(0)

# Iterare attraverso ogni riga di df_137
for idx, row in df_137.iterrows():
    ingredients = row['NER']
    for ingredient in ingredients:
        ingredient = ingredient.strip(',').lower()  
        if ingredient in new_ingredient.columns:
            new_ingredient.at[idx, ingredient] = 1

# Creare il DataFrame finale senza la colonna 'NER'
final_df = pd.concat([df_137['directions'], new_ingredient], axis=1)"""


"new_ingredient = new_ingredient.reindex(index=df_137.index).fillna(0)\n\n# Iterare attraverso ogni riga di df_137\nfor idx, row in df_137.iterrows():\n    ingredients = row['NER']\n    for ingredient in ingredients:\n        ingredient = ingredient.strip(',').lower()  \n        if ingredient in new_ingredient.columns:\n            new_ingredient.at[idx, ingredient] = 1\n\n# Creare il DataFrame finale senza la colonna 'NER'\nfinal_df = pd.concat([df_137['directions'], new_ingredient], axis=1)"

In [141]:
final_df.head(2)

,directions,magnolia_tripetala,calyptranthes_parriculata,chamaecyparis_pisifera_oil,mackerel,mimusops_elengi_flower,hyssop,buchu,black_pepper,eryngium_poterium_oil,...,rue_oil,roasted_coconut,cajeput_oil,star_anise,feijoa,tobacco_oil,cinnamon_leaf,green_tea,artemisia_porrecta_oil,munster_cheese
1517080,"[""In a stockpot, cook the beef, green peppers ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1517081,"[""Place potatoes in a large saucepan and cover...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [142]:
"""# Calculate the number of '1' values for each row in final_df
counts = final_df.iloc[:, 1:].sum(axis=1)  # Consider only the ingredient columns

# Calculate maximum, minimum, and mean of '1' values across all observations
max_1s = counts.max()
min_1s = counts.min()
mean_1s = counts.mean()

# Print the maximum, minimum, and mean of '1' values per row
print("Maximum number of '1' values per row:", max_1s)
print("Minimum number of '1' values per row:", min_1s)
print("Mean number of '1' values per row:", mean_1s)"""

'# Calculate the number of \'1\' values for each row in final_df\ncounts = final_df.iloc[:, 1:].sum(axis=1)  # Consider only the ingredient columns\n\n# Calculate maximum, minimum, and mean of \'1\' values across all observations\nmax_1s = counts.max()\nmin_1s = counts.min()\nmean_1s = counts.mean()\n\n# Print the maximum, minimum, and mean of \'1\' values per row\nprint("Maximum number of \'1\' values per row:", max_1s)\nprint("Minimum number of \'1\' values per row:", min_1s)\nprint("Mean number of \'1\' values per row:", mean_1s)'

In [144]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
ingredient_columns = new_ingredient.columns.tolist()

# Estrai gli ingredienti parziali da df_137['NER']
ingredients_from_ner = []
for idx, row in df_137.iterrows():
    ingredients = row['NER']
    for ingredient in ingredients:
        ingredient = ingredient.strip(',').lower()
        ingredients_from_ner.append(ingredient)

# Calcola la similarità con TF-IDF e trova la migliore corrispondenza
tfidf_vectorizer = TfidfVectorizer(vocabulary=ingredient_columns, lowercase=False)
tfidf_matrix = tfidf_vectorizer.fit_transform(ingredients_from_ner)

# Trova l'indice della colonna con la massima similarità per ogni ingrediente parziale
best_matches = []
for i, ingredient in enumerate(ingredients_from_ner):
    # Calcola la similarità coseno tra l'ingrediente parziale e tutte le colonne
    similarity_scores = cosine_similarity(tfidf_matrix[i], tfidf_vectorizer.transform(ingredient_columns))
    
    # Trova l'indice della colonna con il punteggio di similarità massimo
    best_match_index = similarity_scores.argmax()
    
    # Aggiungi la migliore corrispondenza trovata alla lista
    best_matches.append(ingredient_columns[best_match_index])

# Aggiungi i risultati al DataFrame new_ingredient
for idx, match in enumerate(best_matches):
    new_ingredient.at[idx, match] = 1

# Creare il DataFrame finale senza la colonna 'NER'
final_df = pd.concat([df_137['directions'], new_ingredient], axis=1)

KeyboardInterrupt: 